In [ ]:
%%help

In [59]:
%%cleanup -f

In [60]:
%%configure -f
{"executorMemory": "2000M",
"executorCores": 12,
"numExecutors" : 4}

In [61]:
import pandas as pd
from pprint import pprint
from pyspark import  SparkContext
from pyspark import SparkConf
from pyspark.sql import DataFrame
from IPython.display import display, HTML

deals_df = spark.read.csv("adl://optimumcadence.azuredatalakestore.net/static/deals.csv", header=True)
metarisks_df = spark.read.csv("adl://optimumcadence.azuredatalakestore.net/static/metarisks.csv", header = True)
losses_df = spark.read.parquet("adl://optimumcadence.azuredatalakestore.net/data/v18/calculator/*/parquet/*.parquet")

metarisks_df.createOrReplaceTempView("metarisks")
losses_df.createOrReplaceTempView("losses")
deals_df.createOrReplaceTempView("deals")

#display(deals_df.toPandas())


Starting Spark application


SparkSession available as 'spark'.


In [43]:
#check the ELs match risksrtore output - tells us that the converted eventsets are ok

aal_check_query = """
SELECT DealReference, MetaRiskId, (SUM(LayerAttrition) / 10000) as CalculatedAggEL, ELAgg as ExpectedAggEL, 
ABS((SUM(LayerAttrition) / 10000) - ELAgg) as Delta, ELTID
FROM losses 
INNER JOIN deals ON deals.ELTID = losses.EventSetId 
INNER JOIN metarisks ON metarisks.PerilId = losses.PerilId 
AND metarisks.GeoId = losses.GeoId 
WHERE metarisks.MetaRiskId = 51
GROUP BY DealReference, MetaRiskId, ELAgg, ELTID
ORDER BY Delta DESC
"""

df_aal_check = sqlContext.sql(aal_check_query)
display(df_aal_check.toPandas())

    DealReference MetaRiskId  CalculatedAggEL ExpectedAggEL         Delta  \
0      2751-17-A2         51         0.009357      0.009357  4.994229e-07   
1       2751-17-A         51         0.009357      0.009357  4.994229e-07   
2    1677-17-B2RP         51         0.187295      0.187295  4.991806e-07   
3     1677-17-BRP         51         0.187295      0.187295  4.991806e-07   
4         7562-14         51         0.016458      0.016458  4.888982e-07   
5      1514-17-J1         51         0.217924      0.217924  4.888183e-07   
6       2514-17-E         51         0.003903      0.003903  4.885782e-07   
7      1514-17-F1         51         0.010135      0.010135  4.873449e-07   
8      8854-17-F1         51         0.056745      0.056745  4.871467e-07   
9       3435-17-F         51         0.026551      0.026551  4.822864e-07   
10     1514-17-B1         51         0.109594      0.109594  4.814187e-07   
11      6019-14-A         51         0.016261      0.016261  4.809861e-07   

In [44]:
#calculate ELs for each metarisk

aal_query = """
SELECT DealReference, MetaRiskId, (SUM(LayerAttrition) / 10000) as CalculatedAggEL, ELTID
FROM losses 
INNER JOIN deals ON deals.ELTID = losses.EventSetId 
INNER JOIN metarisks ON metarisks.PerilId = losses.PerilId 
AND metarisks.GeoId = losses.GeoId 
WHERE metarisks.MetaRiskId IN (1, 24, 28, 34, 46, 48, 49, 50, 51, 53)
GROUP BY DealReference, MetaRiskId, ELTID
ORDER BY DealReference, MetaRiskId
"""

df_aal = sqlContext.sql(aal_query)
display(df_aal.toPandas())

     DealReference MetaRiskId  CalculatedAggEL    ELTID
0         10042-17          1     1.056653e-01  2152404
1         10042-17         24     1.697956e-02  2152404
2         10042-17         28     7.589884e-02  2152404
3         10042-17         51     1.056653e-01  2152404
4         10042-17         53     8.868571e-02  2152404
5       10052-17-A          1     1.202431e-02  2158836
6       10052-17-A         24     3.645862e-03  2158836
7       10052-17-A         28     3.825626e-03  2158836
8       10052-17-A         51     1.202431e-02  2158836
9       10052-17-A         53     8.378451e-03  2158836
10        10094-17          1     9.517858e-02  2154798
11        10094-17         24     4.586096e-02  2154798
12        10094-17         51     9.517858e-02  2154798
13        10094-17         53     4.931763e-02  2154798
14      10107-17-A         51     3.427500e-02  2157715
15      10107-17-B         51     5.557500e-02  2157726
16      10107-17-C         51     3.957500e-02  

In [ ]:
# set up deal agg ylts
deal_ylt_query = """
SELECT Simulation, DealReference, ELTID,
SUM(deals.Limit * (CASE deals.DealType WHEN 'B' THEN 1 ELSE -1 END) * losses.LayerAttrition
)  as MetaRiskGrossLoss
,SUM(deals.Limit *(CASE deals.DealType WHEN 'B' THEN 1 ELSE -1 END) *(losses.LayerAttrition - deals.ROL * losses.ReinstatementAttrition)
)  as MetaRiskLoss
,SUM(deals.Limit *(CASE deals.DealType WHEN 'B' THEN -1 ELSE 1 END) * deals.ROL * losses.ReinstatementAttrition
) as MetaRiskReinstatementPremium, MetaRiskId FROM losses 
INNER JOIN deals ON deals.ELTID = losses.EventSetId 
INNER JOIN metarisks ON metarisks.PerilId = losses.PerilId 
AND metarisks.GeoId = losses.GeoId WHERE metarisks.MetaRiskId IN (1, 24, 28, 34, 46, 48, 49, 50, 51, 53)
GROUP BY Simulation, MetaRiskId, DealReference, ELTID
ORDER By Simulation
"""

df_deal_ylt = sqlContext.sql(deal_ylt_query)
df_deal_ylt.createOrReplaceTempView("deal_ylt")

In [ ]:
# apply allocations to deal ylts
%%time

alloc_agg_query = """
SELECT deal_ylt.MetaRiskId, deal_ylt.Simulation, 
SUM(MetaRiskGrossLoss * CCFCronus) as AggGrossCCF,
SUM(MetaRiskLoss * CCFCronus) as AggNetCCF,
SUM(MetaRiskGrossLoss * FCFFerrous) as AggGrossFCF,
SUM(MetaRiskLoss * FCFFerrous) as AggNetFCF,
SUM(MetaRiskGrossLoss * ECFEmerald) as AggGrossECF,
SUM(MetaRiskLoss * ECFEmerald) as AggNetECF,
SUM(MetaRiskGrossLoss * ICFIron) as AggGrossICF,
SUM(MetaRiskLoss * ICFIron) as AggNetICF
FROM
deal_ylt
INNER JOIN
deals
ON deal_ylt.DealReference = deals.DealReference
GROUP BY MetaRiskId, Simulation
ORDER By Simulation, MetaRiskId
"""
df_allocs_agg_deals_ylt = sqlContext.sql(alloc_agg_query)
display(df_allocs_agg_deals_ylt.toPandas())


In [64]:
deal_yelt_query = """
SELECT Simulation, DealReference, ELTID, EventId, EventDay,
SUM(deals.Limit * (CASE deals.DealType WHEN 'B' THEN 1 ELSE -1 END) * losses.LayerAttrition
)  as MetaRiskGrossLoss
,SUM(deals.Limit *(CASE deals.DealType WHEN 'B' THEN 1 ELSE -1 END) *(losses.LayerAttrition - deals.ROL * losses.ReinstatementAttrition)
)  as MetaRiskLoss
,SUM(deals.Limit *(CASE deals.DealType WHEN 'B' THEN -1 ELSE 1 END) * deals.ROL * losses.ReinstatementAttrition
) as MetaRiskReinstatementPremium, MetaRiskId FROM losses 
INNER JOIN deals ON deals.ELTID = losses.EventSetId 
INNER JOIN metarisks ON metarisks.PerilId = losses.PerilId 
AND metarisks.GeoId = losses.GeoId WHERE metarisks.MetaRiskId IN (1, 24, 28, 34, 46, 48, 49, 50, 51, 53)
GROUP BY Simulation, MetaRiskId, DealReference, ELTID, EventId, EventDay
ORDER By Simulation
"""

deal_yelt_query = sqlContext.sql(deal_yelt_query)
deal_yelt_query.createOrReplaceTempView("deal_yelt")
sqlContext.cacheTable("deal_yelt")

In [76]:
# apply allocations to deal ylts

alloc_query = """
SELECT deal_yelt.MetaRiskId, deal_yelt.Simulation, 
MAX(MetaRiskGrossLoss * 'JCF Jupiter') as OccGrossJCF,
MAX(MetaRiskLoss * 'JCF Jupiter') as OccNetJCF,
MAX(MetaRiskGrossLoss * CCFCronus) as OccGrossCCF,
MAX(MetaRiskLoss * CCFCronus) as OccNetCCF,
MAX(MetaRiskGrossLoss * FCFFerrous) as OccGrossFCF,
MAX(MetaRiskLoss * FCFFerrous) as OccNetFCF,
MAX(MetaRiskGrossLoss * ECFEmerald) as OccGrossECF,
MAX(MetaRiskLoss * ECFEmerald) as OccNetECF,
SUM(MetaRiskGrossLoss * CCFCronus) as AggGrossCCF,
SUM(MetaRiskLoss * CCFCronus) as AggNetCCF,
SUM(MetaRiskGrossLoss * FCFFerrous) as AggGrossFCF,
SUM(MetaRiskLoss * FCFFerrous) as AggNetFCF,
SUM(MetaRiskGrossLoss * ECFEmerald) as AggGrossECF,
SUM(MetaRiskLoss * 0.2) as AggNetECF
FROM
deal_yelt
INNER JOIN
deals
ON deal_yelt.DealReference = deals.DealReference
GROUP BY MetaRiskId, Simulation
ORDER By Simulation, MetaRiskId
"""
df_allocs_deals_ylt = sqlContext.sql(alloc_query)
df_allocs_deals_ylt.head(1000)

[Row(MetaRiskId='1', Simulation=1, OccGrossJCF=None, OccNetJCF=None, OccGrossCCF=878680.7628537557, OccNetCCF=878680.7628537557, OccGrossFCF=1653854.3974972824, OccNetFCF=1653854.3974972824, OccGrossECF=2749411.9944202816, OccNetECF=2749411.9944202816, AggGrossCCF=-1614440.551457267, AggNetCCF=-1595680.747376438, AggGrossFCF=-7238165.694319717, AggNetFCF=-7238165.694319717, AggGrossECF=2386764.774869356, AggNetECF=-6399989.163020791), Row(MetaRiskId='24', Simulation=1, OccGrossJCF=None, OccNetJCF=None, OccGrossCCF=229214.8128830386, OccNetCCF=229214.8128830386, OccGrossFCF=124958.28231496393, OccNetFCF=124958.28231496393, OccGrossECF=207734.00652368038, OccNetECF=207734.00652368038, AggGrossCCF=-445860.3745687634, AggNetCCF=-445106.44853002136, AggGrossFCF=-1842244.4119066875, AggNetFCF=-1842244.4119066875, AggGrossECF=77302.80693327803, AggNetECF=-2443747.247841575), Row(MetaRiskId='28', Simulation=1, OccGrossJCF=None, OccNetJCF=None, OccGrossCCF=11718.25859175045, OccNetCCF=11718.258

In [ ]:
#summary_stats = df_ylt['MetaRiskGrossLoss'].sum() / 10000
#pprint(summary_stats)
#df_ylt["Pctile"] = df_ylt.MetaRiskGrossLoss.rank()
#pprint(df_ylt.sort_values("Pctile").head(10))
#df_ylt.plot(x='Pctile', y='MetaRiskGrossLoss', kind='scatter')